# Indoor outdoor classification

In [3]:
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [56]:
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, visualise=vis,
                                  feature_vector=feature_vec, block_norm='L2-Hys')
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, visualise=vis,
                       feature_vector=feature_vec, block_norm='L2-Hys')
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    return np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))

def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    features = []
    for file in imgs:
        image = mpimg.imread(file)
        image = cv2.resize(image, (500, 500))
        file_features = single_img_features(image, color_space=color_space, spatial_size=spatial_size,
                        hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
        
        features.append(file_features)
    return features

In [57]:
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True):   
    
    img_features = []
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    if spatial_feat is True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        img_features.append(spatial_features)
        
    if hist_feat is True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        img_features.append(hist_features)
    if hog_feat is True:
        ch1 = feature_image[:,:,0]
        ch2 = feature_image[:,:,1]
        ch3 = feature_image[:,:,2]
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog_features = np.hstack((hog1, hog2, hog3))
        img_features.append(hog_features.ravel())    

    return np.concatenate(img_features)

In [54]:
facade_path = "./data/train/facade/"
flat_path = "./data/train/flat/"

facades = glob.glob('./data/train/facade/*.jpg')
flats = glob.glob('./data/train/flat/*.jpg')

print(len(facades))
print(len(flats))

36
37


In [44]:
def flip_image(img, path):
    image = mpimg.imread(img)
    image = cv2.flip(image,1)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    name = img.split("/")[-1][:-4]
    cv2.imwrite("{}{}-flip.jpg".format(path, name), image)

for img in flats:
    flip_image(img, flat_path)
    
for img in facades:
    flip_image(img, facade_path)

In [58]:
def stack_array_scale(ones, zeros, colorspace, spatial_size, histbin, orient, pix_per_cell = 8):
    
    one_features = extract_features(ones, color_space=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    
    zeros_features = extract_features(zeros, color_space=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=2,
                        spatial_feat=True, hist_feat=True, hog_feat=True)
    
    # Create an array stack of feature vectors
    X = np.vstack((one_features, zeros_features)).astype(np.float64)
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)

    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    
    return scaled_X, X_scaler

In [85]:
colorspace = 'YUV'
orient = 9
spatial = 32
histbin = 32
spatial_size = (spatial, spatial)

def generate_dataset(ones, zeros, colorspace, spatial_size, histbin, orient):

    # Apply the scaler to X
    scaled_X, X_scaler = stack_array_scale(ones, zeros, colorspace, spatial_size, histbin, orient)

    # Define the labels vector
    y = np.hstack((np.ones(len(ones)), np.zeros(len(zeros))))

    scaled_X, y = shuffle(scaled_X, y)
    
    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
    
    return X_train, X_test, y_train, y_test, X_scaler

X_train, X_test, y_train, y_test, X_scaler = generate_dataset(facades, flats, colorspace,
                                                              spatial_size, histbin, orient)

In [86]:
print(len(X_train))
print(len(X_test))

58
15


In [87]:
print('Using spatial binning of:', spatial,
      'and', histbin, 'histogram bins')
print('Feature vector length:', len(X_train[0]))

rdmf = RandomForestClassifier(max_depth=18, random_state=0)
t = time.time()
#pca = PCA(n_components=300000)
#X_train = pca.fit_transform(X_train)
rdmf.fit(X_train, y_train)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train RandomForest...')
print('Test Accuracy of SVC = {}'.format(rdmf.score(X_test, y_test)))
t = time.time()

print('RandomForest predicts: {}'.format(rdmf.predict(X_test)))
print('y_test labels:         {}'.format(y_test))
t2 = time.time()
print(round(t2 - t, 5), 'Seconds to predict', n_predict, 'labels with SVC')

Using spatial binning of: 32 and 32 histogram bins
Feature vector length: 405036
0.05 Seconds to train RandomForest...
Test Accuracy of SVC = 0.9333333333333333
RandomForest predicts: [ 1.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.]
y_test labels:         [ 1.  1.  0.  0.  0.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.]
0.00716 Seconds to predict 100 labels with SVC


# Results
### With random forest
##### Accuracy
- YCrCb : 0.7727
- YUV : 0.9333
- LUV : 0.7727
- HLS : 0.8636
- HSV : 0.7727
- RGB : 0.7272